# Chapter 01: Use of CELLxGENE

Ray LeClair \<2024-10-23 Wed\>

## Objectives

Since CELLxGENE serves as an initiating data source for the NLM
Knowledge Network pilot, the objectives of this document include
demonstration of:

-   Identification of CELLxGENE datasets for specified organisms, and
    tissues

-   Identification of publications corresponding to CELLxGENE datasets

-   Determination of the dataset filename and dataset file download

### Background

All single-cell RNA data from Chan Zuckerberg (CZ) CELLxGENE Discover
is accessed, queried, and analyzed using the CELLxGENE Discover
Census. Using cell-based slicing and querying one can:

-   Interact with the data through TileDB-SOMA

-   Get slices in AnnData, Seurat, or SingleCellExperiment objects

See: [CELLxGENE Discover Census](https://chanzuckerberg.github.io/cellxgene-census/)

The following sections draw from CZ CELLxGENE tutorials, or a Chan
Zuckerberg Initiative (CZI) repository, which demonstrate how to:

-   [Explore and query the Census in the context of a single tissue, lung](https://chanzuckerberg.github.io/cellxgene-census/notebooks/analysis_demo/comp_bio_explore_and_load_lung_data.html)

-   [Query the expression data and cell/gene metadata from the Census, and load them into common in-memory Python objects](https://chanzuckerberg.github.io/cellxgene-census/notebooks/api_demo/census_query_extract.html)

-   [Generate a citation string for all datasets contained in a Census slice](https://chanzuckerberg.github.io/cellxgene-census/notebooks/api_demo/census_citation_generation.html)

-   [Fetch full metadata for a Dataset](https://github.com/chanzuckerberg/single-cell-curation/blob/0c77179d2e794846861f8109c037b723507959cb/notebooks/curation_api/python_raw/get_dataset.ipynb)

The following sections describe various development environments

See: [springbok-nlm-kn/README.md](https://github.com/ralatsdc/springbok-nlm-kn/blob/main/README.md)

### Jupyter Notebook development environment

Launch Jupyter Notebook from a terminal in which `.zshenv` has been
sourced, and the virtual environment has been activated.

### Emacs Org Mode development environment

Launch Emacs from a terminal in which `.zshenv` has been sourced, then
evaluate this code block to activate the virtual environment:

``` commonlisp
(pyvenv-activate "../../.venv")
```

## Identification of CELLxGENE datasets for specified organisms, and tissues

Mostly following the first tutorial, we write a function that obtains
all datasets, summary cell counts, gene metadata, and, by default,
human and mouse lung, eye, and brain cell metadata from the CZ
CELLxGENE Census as Pandas DataFrames. Anticipating a time consuming
process, the first call of the function writes the DataFrames to
`.parquet` files, then, on subsequent calls, it reads the `.parquet`
files. In both cases, the resulting DataFrames are returned.

To begin, we import modules, and assign module scope variables:

In [1]:
import logging
import os
import re
import subprocess
from time import sleep
from traceback import print_exception

import cellxgene_census

from bs4 import BeautifulSoup
import pandas as pd
import requests

DATA_DIR = "../data"

CELLXGENE_DOMAIN_NAME = "cellxgene.cziscience.com"
CELLXGENE_API_URL_BASE = f"https://api.{CELLXGENE_DOMAIN_NAME}"
CELLXGENE_DIR = f"{DATA_DIR}/cellxgene"

NLM_KN_DIR = f"{DATA_DIR}/nlm-kn"

HTTPS_SLEEP = 1


Next we write the function:

In [2]:
def get_metadata_and_datasets(
    organisms=["homo_sapiens", "mus_musculus"], tissues=["lung", "eye", "brain"]
):
    """Use the CZ CELLxGENE Census to obtain all datasets, summary
    cell counts, gene metadata, and, by default, human and mouse
    lung, eye, and brain cell metadata, then write the resulting
    Pandas DataFrames to parquet files, or, if the files exist, read
    them.

    Parameters
    ----------
    organisms : list(str)
        List of organisms, default is ["homo_sapiens", "mus_musculus"]
    tissues : list(str)
        List of tissues, default is ["lung", "eye", "brain"]

    Returns
    -------
    datasets : pd.DataFrame
        DataFrame containing dataset descriptions
    counts : pd.DataFrame
        DataFrame containing summary cell counts
    var : pd.DataFrame
        DataFrame containing gene metadata
    obs : pd.DataFrame
        DataFrame containing cell metadata
    """
    # Create and write, or read DataFrames
    datasets_parquet = f"{NLM_KN_DIR}/datasets.parquet"
    counts_parquet = f"{NLM_KN_DIR}/counts.parquet"
    var_parquet = f"{NLM_KN_DIR}/var.parquet"
    obs_parquet = f"{NLM_KN_DIR}/obs.parquet"
    if (
        not os.path.exists(datasets_parquet)
        or not os.path.exists(counts_parquet)
        or not os.path.exists(var_parquet)
        or not os.path.exists(obs_parquet)
    ):
        print("Opening soma")
        census = cellxgene_census.open_soma(census_version="latest")

        print("Collecting all datasets")
        datasets = census["census_info"]["datasets"].read().concat().to_pandas()

        print("Collecting summary cell counts")
        counts = (
            census["census_info"]["summary_cell_counts"].read().concat().to_pandas()
        )

        var = pd.DataFrame()
        for organism in organisms:
            print(f"Collecting gene metadata for {organism}")
            var = pd.concat(
                [
                    var,
                    cellxgene_census.get_var(
                        census,
                        organism,
                    ),
                ]
            )

        obs = pd.DataFrame()
        for organism in organisms:
            print(f"Collecting cell metadata for {organism}: {tissues} tissue")
            obs_for_org = cellxgene_census.get_obs(
                census,
                organism,
                value_filter=f"tissue_general in {tissues} and is_primary_data == True",
            )
            obs_for_org["organism"] = organism
            obs = pd.concat(
                [
                    obs,
                    obs_for_org,
                ]
            )

        print("Closing soma")
        census.close()

        print("Writing datasets parquet")
        datasets.to_parquet(datasets_parquet)

        print("Writing summary cell counts parquet")
        counts.to_parquet(counts_parquet)

        print("Writing gene metadata parquet")
        var.to_parquet(var_parquet)

        print("Writing cell metadata parquet")
        obs.to_parquet(obs_parquet)

    else:

        print("Reading datasets parquet")
        datasets = pd.read_parquet(datasets_parquet)

        print("Reading summary cell counts parquet")
        counts = pd.read_parquet(counts_parquet)

        print("Reading gene metadata parquet")
        var = pd.read_parquet(var_parquet)

        print("Reading cell metadata parquet")
        obs = pd.read_parquet(obs_parquet)

    return datasets, counts, var, obs


Then call it to obtain the human and mouse lung, eye, and brain cell
metadata and datasets (using exception handling since accessing an
external resource), and print the result:

In [3]:
try:
    datasets, counts, var, obs = get_metadata_and_datasets()
except Exception as exc:
    print_exception(exc)
print(f"datasets:\n\ncolumns: {datasets.columns}\n\n{datasets}")
print()
print(f"counts:\n\ncolumns: {counts.columns}\n\n{counts}")
print()
print(f"var:\n\ncolumns: {var.columns}\n\n{var}")
print()
print(f"obs:\n\ncolumns: {obs.columns}\n\n{obs}")


Reading datasets parquet
Reading summary cell counts parquet
Reading gene metadata parquet
Reading cell metadata parquet
datasets:

columns: Index(['soma_joinid', 'citation', 'collection_id', 'collection_name',
       'collection_doi', 'collection_doi_label', 'dataset_id',
       'dataset_version_id', 'dataset_title', 'dataset_h5ad_path',
       'dataset_total_cell_count'],
      dtype='object')

     soma_joinid                                           citation  \
0              0  Dataset Version: https://datasets.cellxgene.cz...   
1              1  Publication: https://doi.org/10.1002/hep4.1854...   
2              2  Publication: https://doi.org/10.1126/sciimmuno...   
3              3  Publication: https://doi.org/10.1038/s41593-02...   
4              4  Publication: https://doi.org/10.1038/s41467-02...   
..           ...                                                ...   
951          951  Publication: https://doi.org/10.1101/2023.11.0...   
952          952  Publication: h

We can merge some of the resulting DataFrames to create a summary
DataFrame for export. As before, we will write the summary DataFrame
to a `.parquet` file, so that later we can simply read the `.parquet`
file.

In [4]:
summary_parquet = f"{NLM_KN_DIR}/summary.parquet"
if not os.path.exists(summary_parquet):
    # Define columns needed from obs
    obs_columns = [
        "organism",
        "tissue_general",
        "tissue_general_ontology_term_id",
        "assay",
        "assay_ontology_term_id",
        "dataset_id",
    ]

    # Define columns required for summary
    sum_columns = [
        "organism",
        "tissue_general",
        "tissue_general_ontology_term_id",
        "collection_id",
        "collection_name",
        "collection_doi",
        "assay",
        "assay_ontology_term_id",
        "dataset_id",
        "dataset_title",
        "dataset_h5ad_path",
    ]

    print("Merging datasets and obs DataFrames")
    try:
        summary = pd.merge(
            datasets, obs[obs_columns].drop_duplicates(), on="dataset_id"
        )[sum_columns].drop_duplicates()
    except Exception as exc:
        print_exception(exc)

    print("Writing summary parquet")
    summary.to_parquet(summary_parquet)

    print("Writing summary CSV")
    summary_csv = f"{NLM_KN_DIR}/summary.csv"
    summary.to_csv(summary_csv)

else:
    print("Reading summary parquet")
    summary = pd.read_parquet(summary_parquet)


Reading summary parquet


## Identification of publications corresponding to CELLxGENE datasets

We notice that the datasets DataFrame contains a `citation` column,
for example:

In [5]:
print(datasets["citation"].iloc[4])


Publication: https://doi.org/10.1038/s41467-022-29450-x Dataset Version: https://datasets.cellxgene.cziscience.com/f223d2b0-d878-443b-8bef-f182fe1f22d1.h5ad curated and distributed by CZ CELLxGENE Discover in Collection: https://cellxgene.cziscience.com/collections/bf325905-5e8e-42e3-933d-9a9053e9af80


The `citation` provides the DOI, but not the title of the
publication. Note that we will need the title later in Chapter 02:
E-Utilities. So, we examine the `collection_name` and `dataset_title`
columns:

See: [Chapter-02-E-Utilities.ipynb](Chapter-02-E-Utilities.ipynb)

In [6]:
print(datasets[["collection_name", "dataset_title"]].iloc[4, :])


collection_name    Single-cell Atlas of common variable immunodef...
dataset_title                       Steady-state B cells - scRNA-seq
Name: 4, dtype: object


But it appears we still need to find the title by some method. So, we
write a function that requests the DOI, then parses the resulting
page, most likely from the publisher, to find the title.

In [7]:
def get_title(citation):
    """Get the title given a dataset citation. Note that only wget
    succeeded for Cell Press journals, and neither requests nor wget
    succeeded for The EMBO Journal and Science.

    Parameters
    ----------
    citation : str
        Dataset citation

    Returns
    -------
    title : str
        Title of publication associated with the dataset
    """
    # Need a default return value
    title = None

    # Compile patterns for finding the publication URL and article
    # title
    p1 = re.compile("Publication: (.*) Dataset Version:")
    p2 = re.compile("articleName : '(.*)',")

    # Assign CSS selectors for selecting article title elements
    selectors = [
        "h1.c-article-title",
        "h1.article-header__title.smaller",
        "div.core-container h1",
        "h1.content-header__title.content-header__title--xx-long",
        "h1#page-title.highwire-cite-title",
    ]

    # Find the publication URL
    m1 = p1.search(citation)
    if not m1:
        logging.warning(f"Could not find citation URL for {citation}")
        return title
    citation_url = m1.group(1)
    print(f"Getting title for citation URL: {citation_url}")

    # Attempt to get the publication page using requests
    print(f"Trying requests")
    sleep(HTTPS_SLEEP)
    response = requests.get(citation_url)
    try_wget = True
    if response.status_code == 200:
        html_data = response.text

        # Got the page, so parse it, and try each selector
        fullsoup = BeautifulSoup(html_data, features="lxml")
        for selector in selectors:
            selected = fullsoup.select(selector)
            if selected:

                # Selected the article title, so assign it
                if len(selected) > 1:
                    logging.warning(
                        f"Selected more than one element using {selector} on soup from {citation_url}"
                    )
                title = selected[0].text
                try_wget = False
                break

    if try_wget:

        # Attempt to get the publication page using wget
        print(f"Trying wget")
        sleep(HTTPS_SLEEP)
        completed_process = subprocess.run(
            ["curl", "-L", citation_url], capture_output=True
        )
        html_data = completed_process.stdout

        # Got the page, so parse it, and search for the title
        fullsoup = BeautifulSoup(html_data, features="lxml")
        found = fullsoup.find_all("script")
        if found and len(found) > 4:
            m2 = p2.search(found[4].text)
            if m2:
                title = m2.group(1)

    print(f"Found title: '{title}' for citation URL: {citation_url}")

    return title


Next we call the function for an example citation (again using
exception handling since accessing an external resource):

In [8]:
try:
    citation = datasets["citation"].iloc[4]
    title = get_title(citation)
except Exception as exc:
    print_exception(exc)


Getting title for citation URL: https://doi.org/10.1038/s41467-022-29450-x
Trying requests
Found title: 'Single-cell Atlas of common variable immunodeficiency shows germinal center-associated epigenetic dysregulation in B-cell responses' for citation URL: https://doi.org/10.1038/s41467-022-29450-x


Note that the function attempts to use `requests`, and if it fails,
`wget`, since some publishers respond to one, but not the other. The
selectors were discovered by manually inspecting the pages returned
for the human lung cell datasets using Google Chrome Developer Tools.

## Determine the dataset filename and download the dataset file.

Following a notebook found in a CZI repository, we write a function to
find the dataset filename, and to download the dataset file, given a
row of the datasets DataFrame obtained above:

In [9]:
def get_and_download_dataset_h5ad_file(dataset_series):
    """Get the dataset filename and download the dataset file.

    Parameters
    ----------
    dataset_series : pd.Series
        A row from the dataset DataFrame

    Returns
    -------
    dataset : str
       The dataset filename
    """
    # Need a default return value
    dataset_filename = None

    # Get the dataset object
    collection_id = dataset_series.collection_id
    dataset_id = dataset_series.dataset_id
    dataset_url = f"{CELLXGENE_API_URL_BASE}/curation/v1/collections/{collection_id}/datasets/{dataset_id}"
    sleep(HTTPS_SLEEP)
    response = requests.get(dataset_url)
    response.raise_for_status()
    if response.status_code != 200:
        logging.error(f"Could not get dataset for id {dataset_id}")
        return

    data = response.json()
    if dataset_id != data["dataset_id"]:
        logging.error(
            f"Response dataset id: {data['dataset_id']} does not equal specified dataset id: {dataset_id}"
        )
        return

    # Find H5AD files, if possible
    assets = data["assets"]
    for asset in assets:
        if asset["filetype"] != "H5AD":
            continue

        # Found an H5AD file, so download it, if needed
        dataset_filename = f"{dataset_id}.{asset['filetype']}"
        dataset_filepath = f"{CELLXGENE_DIR}/{dataset_filename}"
        if not os.path.exists(dataset_filepath):
            print(f"Downloading dataset file: {dataset_filepath}")
            with requests.get(asset["url"], stream=True) as response:
                response.raise_for_status()
                with open(dataset_filepath, "wb") as df:
                    for chunk in response.iter_content(chunk_size=1024 * 1024):
                        df.write(chunk)
            print(f"Dataset file: {dataset_filepath} downloaded")

        else:
            print(f"Dataset file: {dataset_filepath} exists")

    return dataset_filename


Then call it using the first row of the datasets DataFrame obtained
above, and print the result (we'll use exception handling when
accessing an external resource from now on):

In [10]:
try:
    dataset_series = datasets.iloc[4]
    get_and_download_dataset_h5ad_file(dataset_series)
except Exception as exc:
    print_exception(exc)


Dataset file: ../data/cellxgene/a5d95a42-0137-496f-8a60-101e17f263c8.H5AD exists


Next, in Chapter 02 we write functions to search PubMed for the title
and identifiers.

See: [Chapter-02-E-Utilities.ipynb](Chapter-02-E-Utilities.ipynb)